In [ ]:
import datetime
import glob
import os
import earthaccess
import numpy as np
import h5py
import py4eos
from matplotlib import pyplot

auth = earthaccess.login()

VNP16_DIR = 'data/VNP16A2GF'
SMAP_DIR = 'data/SMAP_SM_L3'

```python
help(earthaccess.search_data)
```

```
**bounding_box**: a tuple representing spatial bounds in the form
    `(lower_left_lon, lower_left_lat, upper_right_lon, upper_right_lat)`
```

In [ ]:
bbox = (1.5, 34.0, 8.0, 37.0)

In [ ]:
results = earthaccess.search_data(
    short_name = 'VNP16A2GF',
    temporal = ('2023-01-01', '2023-12-31'),
    bounding_box = tuple(bbox))

In [ ]:
# "Check that we're getting N results for N dates"

len(results)

In [ ]:
# Only download the files once; i.e., if we haven't already downloaded any
if len(glob.glob(f'{VNP16_DIR}/*')) == 0:
    earthaccess.download(results, VNP16_DIR)

In [ ]:
hdf = py4eos.read_hdf4eos('data/VNP16A2GF/VNP16A2GF.A2023001.h18v05.002.2025003083850.h5', platform = 'VIIRS')
hdf

In [ ]:
# TODO Note scale factors and valid min/max

et0 = hdf.get('ET_500m')
pet0 = hdf.get('PET_500m')
et = np.where(np.abs(et0) >= 32700, np.nan, et0 * 0.1)
pet = np.where(np.abs(pet0) >= 32700, np.nan, pet0 * 0.1)

## TODO Adding ET data processing to SnakeMake

In [ ]:
results = earthaccess.search_data(
    short_name = 'VNP16A2GF',
    temporal = ('2023-01-01', '2023-12-31'),
    bounding_box = tuple(bbox))

## Initializing soil water content using satellite soil moisture data

DOI: https://dx.doi.org/10.5067/M20OXIZHY3RJ

In [ ]:
# NOTE: We only need the first few days, to initialize soil water

results = earthaccess.search_data(
    short_name = 'SPL3SMP_E',
    temporal = ('2023-01-01', '2023-01-10'))

In [ ]:
# Only download the files once; i.e., if we haven't already downloaded any
if len(glob.glob(f'{SMAP_DIR}/*')) == 0:
    earthaccess.download(results, SMAP_DIR)

In [ ]:
# NOTE: We're going to stack the AM and PM data together for each day,
#    then take the mean over the first 10 days

stack = []
for filename in glob.glob(f'{SMAP_DIR}/*.h5'):
    with h5py.File(filename, 'r') as hdf:
        sm_am = hdf['Soil_Moisture_Retrieval_Data_AM/soil_moisture_dca'][:]
        sm_pm = hdf['Soil_Moisture_Retrieval_Data_PM/soil_moisture_dca_pm'][:]
        sm_am[sm_am < 0] = np.nan
        sm_pm[sm_pm < 0] = np.nan
        sm = np.where(np.isnan(sm_am), sm_pm, sm_am)
        stack.append(sm)

sm = np.nanmean(np.stack(stack, axis = 0), axis = 0)

In [ ]:
pyplot.imshow(sm, interpolation = 'nearest')